# LLM-Langchain

## Init

### Libraries

In [2]:
import datetime
from dotenv import load_dotenv
import os

load_dotenv()

# LLM AWS
import boto3
from langchain_aws.chat_models.bedrock import ChatBedrock

# LLM Azure
from langchain_openai import AzureChatOpenAI

# Tools
from langchain_core.tools import tool

# Prompt
from langchain_core.prompts import PromptTemplate

# Output Parser
from langchain_core.output_parsers import JsonOutputParser

### LLMs

In [3]:
## Claude - AWS
AWS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY')
AWS_SECRET_KEY = os.getenv('AWS_SECRET_KEY')
REGION_NAME = os.getenv('REGION_NAME')

boto3.setup_default_session(aws_access_key_id=AWS_ACCESS_KEY,
                            aws_secret_access_key=AWS_SECRET_KEY,
                            region_name=REGION_NAME)

bedrock_client = boto3.client(service_name='bedrock-runtime')

llm_claude = ChatBedrock(
    credentials_profile_name="bedrock-admin",
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    client=bedrock_client
)

In [4]:
## GPT4o-mini - Azure
AZURE_APIKEY = os.environ["AZURE_OPENAI_APIKEY"]
AZURE_DEPLOYMENT = os.environ["AZURE_OPENAI_DEPLOYMENT"]
AZURE_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]

llm_openai = AzureChatOpenAI(
    deployment_name= AZURE_DEPLOYMENT,
    api_version="2023-03-15-preview",
    api_key= AZURE_APIKEY,
    azure_endpoint= AZURE_ENDPOINT,
    openai_api_type= "azure",
    temperature= 0
)

In [ ]:
## Gemini-flash-2 - GCP

### Chain

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step.

In [5]:
# Prompt
prompt_related_word = PromptTemplate.from_template("""
Recibirás una lista de palabras, que pueden o no relacionarse entre ellas.
Debes generar una palabra que se relacione con las que te entregué y se pueda incluir en la lista haciendo que mantenga el sentido.

Las palabras son:
{palabras}

La respuesta debes entregármela como un JSON:
respuesta: Es la respuesta que encontraste
justificacion: Justificación corta de por qué elegiste esa palabra
""")

prompt_related_word.format(palabras=["ají","merkén","chile"])

"\nRecibirás una lista de palabras, que pueden o no relacionarse entre ellas.\nDebes generar una palabra que se relacione con las que te entregué y se pueda incluir en la lista haciendo que mantenga el sentido.\n\nLas palabras son:\n['ají', 'merkén', 'chile']\n\nLa respuesta debes entregármela como un JSON:\nrespuesta: Es la respuesta que encontraste\njustificacion: Justificación corta de por qué elegiste esa palabra\n"

In [6]:
# Parser
parser_json = JsonOutputParser()

In [8]:
chain = prompt_related_word | llm_claude | parser_json
print("Claude:", chain.invoke({"palabras":["ají","merkén","chile"]}))

chain = prompt_related_word | llm_openai | parser_json
print("OpenAI:", chain.invoke({"palabras":["ají","merkén","chile"]}))

Claude: {'respuesta': 'picante', 'justificacion': "La palabra 'picante' se relaciona con las palabras 'ají', 'merkén' y 'chile', ya que todas ellas se refieren a tipos de especias o condimentos que se caracterizan por su sabor picante."}
OpenAI: {'respuesta': 'pimiento', 'justificacion': 'El pimiento es un término general que se relaciona con el ají y el chile, ya que ambos son tipos de pimientos picantes.'}


### Tools

In [9]:
horas = {1,2,3,19,22}
total = {0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,21,22,23,24}

@tool
def agendar_hora(hora: int):
    """
    Recibe una hora para agendar, que va desde 0 a 24 y agenda la hora
    """
    return horas.add(hora)

@tool
def disponibilidad_horaria(hora: int):
    """
    Recibe una hora, que va desde 0 a 24 y consulta en la agenda para ver si está disponible
    """
    if hora in horas:
        return False
    return True

@tool
def listar_horas_disponibles():
    """
    Lista las horas disponibles para agendar
    """
    return total - horas



def agendar_hora_func(hora: int):
    """
    Recibe una hora para agendar, que va desde 0 a 24 y agenda la hora
    """
    return horas.add(hora)


def disponibilidad_horaria_func(hora: int):
    """
    Recibe una hora, que va desde 0 a 24 y consulta en la agenda para ver si está disponible
    """
    if hora in horas:
        return False
    return True


def listar_horas_disponibles_func():
    """
    Lista las horas disponibles para agendar
    """
    return total - horas


In [10]:
tools = [agendar_hora, disponibilidad_horaria, listar_horas_disponibles]
functions = {
    "agendar_hora": agendar_hora,
    "disponibilidad_horaria": disponibilidad_horaria,
    "listar_horas_disponibles": listar_horas_disponibles
}

llm_tools = llm_claude.bind_tools(tools)
llm_tools

RunnableBinding(bound=ChatBedrock(client=<botocore.client.BedrockRuntime object at 0x000001A2414D6E50>, credentials_profile_name='bedrock-admin', model_id='anthropic.claude-3-haiku-20240307-v1:0'), kwargs={'tools': [{'name': 'agendar_hora', 'description': 'Recibe una hora para agendar, que va desde 0 a 24 y agenda la hora', 'input_schema': {'properties': {'hora': {'type': 'integer'}}, 'required': ['hora'], 'type': 'object'}}, {'name': 'disponibilidad_horaria', 'description': 'Recibe una hora, que va desde 0 a 24 y consulta en la agenda para ver si está disponible', 'input_schema': {'properties': {'hora': {'type': 'integer'}}, 'required': ['hora'], 'type': 'object'}}, {'name': 'listar_horas_disponibles', 'description': 'Lista las horas disponibles para agendar', 'input_schema': {'properties': {}, 'type': 'object'}}]}, config={}, config_factories=[])

In [ ]:
while True:
    # Imprime horas
    horas_sort = list(horas)
    horas_sort.sort()
    print("\033[32m","Horas en uso:",horas_sort)

    # Recibe entrada
    print("\033[35mInput:",end="")
    res = input("")
    print(f"\033[35m",res)

    messages = [
    ("human", res),
    ]

    ans = llm_tools.invoke(messages)
    

    if (ans.tool_calls):
        for call in ans.tool_calls:
            
            function_name = call['name']
            function_args = call['args']

            if function_name == "listar_horas_disponibles":
                print(listar_horas_disponibles_func())
            elif function_name == "disponibilidad_horaria":
                print(disponibilidad_horaria_func(function_args['hora']))
            elif function_name == "agendar_hora":
                agendar_hora_func(function_args['hora'])
                print(f"agendó una hora '{function_args['hora']}'")
            print()
    else:
        print(ans.content)
        print()

